In [ ]:
import xarray as xr
import numpy as np
import geopandas as gp

from nb_shared import *

from swe_compare.nb_helpers import start_cluster
from swe_compare import CBRFCZone, Snow17SWE, ZoneCompare
from swe_compare.rasterize_zone import cbrfc_zone_mask_as_xr

# Description

Notebook to spatially join a rasterized CBRFC zone with the centers of the grid cells of a SWANN data set.  
This was not successful as the centers do not all overlap with a zone polygon at that location.

In [ ]:
upperColorado = CBRFCZone(UC_ZONES)
cbrfc_zones = cbrfc_zone_mask_as_xr(UC_ZONE_TIF, upperColorado)

In [ ]:
s17_swe = Snow17SWE(
    SWE_HOME_DIR + 'CBRFC_Gunnison_1981_2020_SWE_inches.csv',
)
target_zones = list(s17_swe.csv.columns)
target_zones = upperColorado.target_zones_as_dict(target_zones)
target_zones

In [ ]:
alec2_huf = upperColorado.shapefile.query("zone == 'ALEC2HUF'")

In [ ]:
alec2_huf.bounds

In [ ]:
swann = xr.open_mfdataset(
    SWANN_HOME_DIR + '*2019_v01.nc',
)

In [ ]:
swann = swann.SWE.sel(
    lat=slice(alec2_huf.bounds.miny.values[0], alec2_huf.bounds.maxy.values[0]),
    lon=slice(alec2_huf.bounds.minx.values[0], alec2_huf.bounds.maxx.values[0]),
)

In [ ]:
df = swann.isel(time=[0]).to_dataframe().reset_index()
df

In [ ]:
swann_gp = gp.GeoDataFrame(
    df, geometry=gp.points_from_xy(df.lon,df.lat, crs="EPSG:4269")
)

swann_gp

In [ ]:
alec2_swann_swe = alec2_huf.sjoin(swann_gp)

In [ ]:
alec2_swann_swe